In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
from sklearn import preprocessing
from sklearn import datasets

In [23]:
INPUT_PATH = "D:\\Northeastern\\100 pct undersample split\\"
split = "test"

X = pd.read_csv(INPUT_PATH + f"X_{split}.csv")

# Gender
We treat 'unknown' as a 3rd category of gender with the assumption being that the user's decision to not provide gender may be relevant to our predictions

In [24]:
X.loc[X["gender"] == "male","gender"] = "M"
X.loc[X["gender"] == "female","gender"] = "F"
X.loc[:,"gender"]=X["gender"].fillna("U")

enc_gender = preprocessing.OneHotEncoder(drop="first",categories="auto")
gender_onehot = enc_gender.fit_transform(X.gender.values.reshape(-1,1)).toarray()
# Drop old gender column and merge new onehot encoded columns
X=X.drop(["gender"],axis=1)
X["gender_male"] = gender_onehot[:,0]
X["gender_unknown"] = gender_onehot[:,1]
X.head()

,msno,city,bd,registered_via,registration_init_time,gender_male,gender_unknown
0,VLmKab6JByryjhBzawWgIXnsDn/p8gtuKNUDEIrALIw=,8,33,3,20111215,1.0,0.0
1,3rlAaL9Cf69Pq+Rcvdckx4D0SDTFILEeRLfxfdYxY1U=,13,23,9,20130312,1.0,0.0
2,9O0C/OkJj+j+FZbSFgc/Q/VaopXYXfEBTZ7kUVKSJRg=,1,0,7,20150305,0.0,1.0
3,ie5tsyNQ7rguYSr3TqLLH19l13Z3cfCcLMCqWQyTK9w=,13,29,9,20071022,1.0,0.0
4,sfONs5t/KwmWNA3dejHla3sZtltkSvXHyTscrXZxgls=,6,44,9,20140218,0.0,0.0


# Registration Date
We convert registration date into an integer containing the number of days since registration. Those values are then standardized.

In [25]:
reg_date = pd.to_datetime(X.registration_init_time, format="%Y%m%d")
# Assuming here that our training date for predicting February 2017 is Jan 1, 2017
train_date = pd.to_datetime("20170131", format="%Y%m%d")
reg_days = (train_date - reg_date).dt.days

reg_days_scaled = preprocessing.scale(reg_days)
X["reg_days_scaled"] = pd.Series(reg_days_scaled)
X=X.drop(["registration_init_time"], axis=1)
X.head()

,msno,city,bd,registered_via,gender_male,gender_unknown,reg_days_scaled
0,VLmKab6JByryjhBzawWgIXnsDn/p8gtuKNUDEIrALIw=,8,33,3,1.0,0.0,0.315971
1,3rlAaL9Cf69Pq+Rcvdckx4D0SDTFILEeRLfxfdYxY1U=,13,23,9,1.0,0.0,-0.122111
2,9O0C/OkJj+j+FZbSFgc/Q/VaopXYXfEBTZ7kUVKSJRg=,1,0,7,0.0,1.0,-0.821300
3,ie5tsyNQ7rguYSr3TqLLH19l13Z3cfCcLMCqWQyTK9w=,13,29,9,1.0,0.0,1.781077
4,sfONs5t/KwmWNA3dejHla3sZtltkSvXHyTscrXZxgls=,6,44,9,0.0,0.0,-0.453815


# Registration Method
We one-hot encode the registration methods

In [26]:
enc_registration = preprocessing.OneHotEncoder(drop="first",categories="auto")
registration_onehot = enc_registration.fit_transform(X.registered_via.values.reshape(-1,1)).toarray()
registration_df = pd.DataFrame(registration_onehot)
registration_df.columns = enc_registration.get_feature_names(["registered_via"])
X = pd.concat([X,registration_df], axis=1)
X = X.drop(["registered_via"],axis=1)
X.head()

,msno,city,bd,gender_male,gender_unknown,reg_days_scaled,registered_via_4,registered_via_7,registered_via_9
0,VLmKab6JByryjhBzawWgIXnsDn/p8gtuKNUDEIrALIw=,8,33,1.0,0.0,0.315971,0.0,0.0,0.0
1,3rlAaL9Cf69Pq+Rcvdckx4D0SDTFILEeRLfxfdYxY1U=,13,23,1.0,0.0,-0.122111,0.0,0.0,1.0
2,9O0C/OkJj+j+FZbSFgc/Q/VaopXYXfEBTZ7kUVKSJRg=,1,0,0.0,1.0,-0.821300,0.0,1.0,0.0
3,ie5tsyNQ7rguYSr3TqLLH19l13Z3cfCcLMCqWQyTK9w=,13,29,1.0,0.0,1.781077,0.0,0.0,1.0
4,sfONs5t/KwmWNA3dejHla3sZtltkSvXHyTscrXZxgls=,6,44,0.0,0.0,-0.453815,0.0,0.0,1.0


# City
We one-hot encode the city

In [27]:
enc_city = preprocessing.OneHotEncoder(drop="first",categories="auto")
city_onehot = enc_city.fit_transform(X.city.values.reshape(-1,1)).toarray()
city_df = pd.DataFrame(city_onehot)
city_df.columns = enc_city.get_feature_names(["city"])
X = pd.concat([X,city_df], axis=1)
X = X.drop(["city"],axis=1)
X.head()

,msno,bd,gender_male,gender_unknown,reg_days_scaled,registered_via_4,registered_via_7,registered_via_9,city_3,city_4,...,city_12,city_13,city_14,city_15,city_16,city_17,city_18,city_20,city_21,city_22
0,VLmKab6JByryjhBzawWgIXnsDn/p8gtuKNUDEIrALIw=,33,1.0,0.0,0.315971,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3rlAaL9Cf69Pq+Rcvdckx4D0SDTFILEeRLfxfdYxY1U=,23,1.0,0.0,-0.122111,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9O0C/OkJj+j+FZbSFgc/Q/VaopXYXfEBTZ7kUVKSJRg=,0,0.0,1.0,-0.821300,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ie5tsyNQ7rguYSr3TqLLH19l13Z3cfCcLMCqWQyTK9w=,29,1.0,0.0,1.781077,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,sfONs5t/KwmWNA3dejHla3sZtltkSvXHyTscrXZxgls=,44,0.0,0.0,-0.453815,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Birthday

Scaling the BD values. Others have indicated that this feature is predictive (despite the strange, negative values)

In [28]:
X["bd"] = preprocessing.scale(X.bd)
X.head()

,msno,bd,gender_male,gender_unknown,reg_days_scaled,registered_via_4,registered_via_7,registered_via_9,city_3,city_4,...,city_12,city_13,city_14,city_15,city_16,city_17,city_18,city_20,city_21,city_22
0,VLmKab6JByryjhBzawWgIXnsDn/p8gtuKNUDEIrALIw=,0.953129,1.0,0.0,0.315971,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3rlAaL9Cf69Pq+Rcvdckx4D0SDTFILEeRLfxfdYxY1U=,0.325388,1.0,0.0,-0.122111,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9O0C/OkJj+j+FZbSFgc/Q/VaopXYXfEBTZ7kUVKSJRg=,-1.118415,0.0,1.0,-0.821300,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ie5tsyNQ7rguYSr3TqLLH19l13Z3cfCcLMCqWQyTK9w=,0.702033,1.0,0.0,1.781077,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,sfONs5t/KwmWNA3dejHla3sZtltkSvXHyTscrXZxgls=,1.643643,0.0,0.0,-0.453815,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
X.to_csv(INPUT_PATH + f"X_{split}_transformed.csv",index=False)